In [103]:
from openai import OpenAI
import json

client = OpenAI()

# Define the Python functions
def get_weather(latitude,longitude):
    return f"The current weather in {latitude} and {longitude} is sunny and 25°C."

def get_time(time):
    return f"The current time is {time}"

# Map function names to actual Python functions
function_map = {
    "get_weather": get_weather,
    "get_time": get_time
}


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for provided coordinates in Celsius.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"}
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False
            },
            "strict": True
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "Get the current time for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "time": {
                        "type": "string",
                        "description": "City and country, e.g., Tokyo, Japan"
                    }
                },
                "required": ["time"],  # Corrected the required key to match the properties
                "additionalProperties": False
            },
            "strict": True
        }
    }
]


def ask_gpt(msg):
    messages = [{"role": "user", "content": msg}]
    
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
    )
    
    tool_call = completion.choices[0].message.tool_calls
    print(tool_call)
    return tool_call


In [104]:
print(completion.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_NA2F5Ar4UHJuV7wjYWpihchq', function=Function(arguments='{"latitude":48.8566,"longitude":2.3522}', name='get_weather'), type='function')]


In [105]:
tool_calls = ask_gpt('what is the time in japan')

[ChatCompletionMessageToolCall(id='call_bUXyV4BtCJesqXcKHocKiBj9', function=Function(arguments='{"time":"Tokyo, Japan"}', name='get_time'), type='function')]


In [106]:
tool_calls[0].function.name

'get_time'

In [107]:
tool_calls = ask_gpt('i have coodinates like 10 and 20 get temperature')

if tool_calls:
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        tool_arguments = json.loads(tool_call.function.arguments)
    
        # Dynamically call the appropriate function
        if tool_name in function_map:
            result = function_map[tool_name](**tool_arguments)  # Call the mapped function with arguments
            print(f"Result: {result} ")
        else:
            print(f"Function '{tool_name}' is not implemented.")

[ChatCompletionMessageToolCall(id='call_wzpSMdvuLVZcoEDsgm7brPf6', function=Function(arguments='{"latitude":10,"longitude":20}', name='get_weather'), type='function')]
Result: The current weather in 10 and 20 is sunny and 25°C. 
